## Load necessary modules

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import re

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('retinanet_models', 'resnet50_coco_best_v2.1.0.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.load_model(model_path, backbone_name='resnet50', convert_model=True)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

## Run detection on example

## Tightly Crop Cars

In [ ]:
for root, dirs, files in os.walk('cars'):
    # skip empty folders
    if len(files) == 0:
        continue
    
    print(root, ":", len(files), "files")
    
    start = time.time()
    
    # create cropped images folder
    cropped_root = re.sub(r"^cars", "cars_cropped", root)
    if not os.path.exists(cropped_root):
        os.makedirs(cropped_root)
    
    fig = plt.figure(figsize=(15, 15))
    
    # os.walk() returns file list as unsorted
    for inx, name in enumerate(sorted(files)):
        
        # load image
        image = read_image_bgr(os.path.join(root, name))
        
        # copy to draw on
        draw = image.copy()
        
        # preprocess image for network
        image = preprocess_image(image)
        image, scale = resize_image(image)
        
        # process image
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        
        # correct for image scale
        boxes /= scale
        
        # Be sure to get the first car detection
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            # ignore non-cars
            if label != 2:
                continue
            else: # car is found
                break
        # box, score, label is accessible in this scope
        
        b = box.round().astype(int)
        cropped = draw[b[1]:b[3]+1, b[0]:b[2]+1, :]
        
        cv2.imwrite(os.path.join(cropped_root, name), cropped)
        
        ax = plt.subplot(9, 8, inx+1)
        # OpenCV uses BGR. BGR->RGB conversion is required for matplotlib
        ax.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        ax.axis('off')
    
    print("processing time: ", time.time() - start)
    
    fig.savefig("{}{}".format(cropped_root, ".png"))
    #plt.show()